In [1]:
import numpy as np
import pandas as pd

# Parameters Vector

Load, merge, normalization and visualisation vector of parameters.

In [18]:
# Loading each csv file
df1 = pd.read_csv('malehealthy.csv')
df2 = pd.read_csv('malepathology.csv')
df3 = pd.read_csv('femalehealthy.csv')
df4 = pd.read_csv('femalepathology.csv')

Concatenating all four DataFrames into one.

In [19]:
allpatients = pd.concat([df1,df2,df3,df4])
allpatients.head()

,STD,MEAN,MAX,MIN,RMS,ENERGY,POWER,Status
0,6289.628284,10.762239,29241.0,-29320.0,6289.637492,3.142491e+12,1.977965e+07,healthy
1,5107.064228,-8.317314,22856.0,-29453.0,5107.071001,2.332659e+12,1.304101e+07,healthy
2,6599.732595,26.978027,22061.0,-23299.0,6599.787734,1.791987e+12,2.177833e+07,healthy
3,3353.416152,-16.966091,9884.0,-13521.0,3353.459070,8.231281e+11,5.622805e+06,healthy
4,4251.474562,84.546723,13997.0,-19188.0,4252.315146,1.523442e+12,9.041038e+06,healthy


Features has diffrent rows, so they have to be normalized.

In [20]:
features = ['STD','MEAN','MAX','MIN','RMS','ENERGY','POWER']
for i in features:
    allpatients[i] = (allpatients[i] - np.mean(allpatients[i]))/np.std(allpatients[i])
allpatients.head()

,STD,MEAN,MAX,MIN,RMS,ENERGY,POWER,Status
0,0.427009,0.166092,2.282855,-1.734309,0.411114,0.642207,0.251356,healthy
1,-0.252270,0.131723,1.098892,-1.754664,-0.270637,0.117862,-0.369856,healthy
2,0.605136,0.195303,0.951476,-0.812862,0.589916,-0.232209,0.435609,healthy
3,-1.259587,0.116143,-1.306491,0.683552,-1.281596,-0.859520,-1.053716,healthy
4,-0.743731,0.299006,-0.543822,-0.183720,-0.763404,-0.406085,-0.738600,healthy


# Detection alghoritm

Logistic regression

# Train test split

# Training and predicting

In [67]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logModel = LogisiticRegression()
logModel.fit(xTrain, yTrain)